In [568]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

## Importing part

In [634]:
train = pd.read_csv('train_ctrUa4K.csv')
test = pd.read_csv('test_lAUu6dG.csv')
sample_submission = pd.read_csv('sample_submission_49d68Cx.csv')

target = train['Loan_Status']
train.drop(['Loan_Status'], axis=1, inplace=True)

idx = len(train)

In [635]:
train_num = train.select_dtypes(np.number)
test_num = test.select_dtypes(np.number)
num = pd.concat([train_num, test_num], axis=0).reset_index(drop=True)

train_cat = train.select_dtypes(include='object')
test_cat = test.select_dtypes(include='object')
cat = pd.concat([train_cat, test_cat], axis=0).reset_index(drop=True)

## Feature engineering

In [636]:
num_tr = num.copy()
cat_tr = cat.copy()

**Перед заполением нулей избавимся от выбросов в числовых признаках**

In [637]:
for col in ['ApplicantIncome', 'LoanAmount']:

      Q1 = np.nanpercentile(num_tr[col], 25)
      Q3 = np.nanpercentile(num_tr[col], 75)
      IQR = Q3 - Q1
      
      num_tr['{}_Outliers'.format(col)] = num_tr[col].map(lambda x: 'Normal' if x <= Q3 + 1.5 * IQR else 'Outlier') 
      # # temp[col > Q3 + 1.5 * IQR].groupby(col).mean()['Loan_Status']
      # print(temp[temp[col] > Q3 + 1.5 * IQR]['Loan_Status'].mean())

num_tr['CoapplicantIncome_Outliers'] = num_tr['CoapplicantIncome'].map(lambda x: 'Normal' if x <= 6700 else 'Outlier')

**Заполнение нулей**

In [638]:
num_tr['LoanAmount'] = num_tr['LoanAmount'].fillna(num_tr['LoanAmount'].mean())
num_tr['Loan_Amount_Term'] = num_tr['Loan_Amount_Term'].fillna(360.00)

#Знаю, что плохо, но это временно 
num_tr['Credit_History'] = num_tr['Credit_History'].fillna(0.5)

In [639]:
#Cat dummies - временная, дальше исп не будут
#В целом все действия тут нужны, чтобы предсказать только пропущеную кредитную историю
#Тут хитро: 
#         -> LoanAmount, 
#         потом уже с LoanAmount -> Credit_History

for col in ['Gender', 'Married', 'Dependents', 'Education',
            'Self_Employed', 'Property_Area']:
  cat[col] = cat[col].fillna(cat[col].mode()[0])


cat_dummies = pd.DataFrame()
for col in ['Gender', 'Married', 'Dependents', 'Education',
            'Self_Employed', 'Property_Area']:
  cat_dummies = pd.concat([cat_dummies, pd.get_dummies(data=cat[col], drop_first=True, prefix=col)], axis=1)

for col in ['ApplicantIncome_Outliers', 'CoapplicantIncome_Outliers', 'LoanAmount_Outliers']:
  cat_dummies = pd.concat([cat_dummies, pd.get_dummies(data=num_tr[col], drop_first=True, prefix=col)], axis=1)


sc = MinMaxScaler()

X_ch = pd.concat([num_tr.drop(['ApplicantIncome_Outliers', 'CoapplicantIncome_Outliers', 'LoanAmount_Outliers'], axis=1), cat_dummies], axis=1)
X_ch = pd.DataFrame(data=sc.fit_transform(X_ch), columns=X_ch.columns)




rf = RandomForestClassifier(n_estimators=100, 
                            max_features=6, oob_score=True)

rf.fit(X_ch.iloc[:idx, :], target.map({'Y': 0, 'N': 1}))
X_ch['Loan_Status_temp'] = np.hstack((target.map({'Y': 0, 'N': 1}), rf.predict(X_ch.iloc[idx:, :])))



RS=4

ch = X_ch['Credit_History']
X_ch = X_ch.drop(['Credit_History'], axis=1)

ch_to_train_test =  ch[ch!=0.5]
X_ch_to_train_test =  X_ch[ch!=0.5]


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_ch_to_train_test, ch_to_train_test);


X_ch['Credit_History'] = ch
X_ch['Credit_History'][X_ch['Credit_History'] == 0.5] = knn.predict(X_ch.drop(['Credit_History'], axis=1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [640]:
#Finally
num_tr['Credit_History'] = X_ch['Credit_History']

**"Настоящее" заполнение нулей для категориональных**

In [641]:
for col in ['Gender', 'Married', 'Dependents', 'Education',
            'Self_Employed', 'Property_Area']:
  cat_tr[col] = cat_tr[col].fillna(cat_tr[col].mode()[0])

**Соединяем числовые и тренировочные фичи**

In [642]:
all_data_tr = pd.concat([num_tr, cat_tr], axis=1)

**Теперь генерим фичи**

In [643]:
all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Loan_Amount_Term'].copy()

all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Group_Loan_Amount_Term'].replace([6.0, 12.0, 36.0, 60.0, 84.0, 120.0], '<= 10 years')
all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Group_Loan_Amount_Term'].replace(180.0, '15 years')
all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Group_Loan_Amount_Term'].replace(360.0, '30 years')
all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Group_Loan_Amount_Term'].replace([240.0, 300.0, 350.0], 'Not beautiful dates')
all_data_tr['Group_Loan_Amount_Term'] = all_data_tr['Group_Loan_Amount_Term'].replace([480.0], '40 years')

all_data_tr.drop(['Loan_Amount_Term'], axis=1, inplace=True)

In [644]:
# Наша рандомная фича, извлекаем "цифру" и переводим в числовой формат 
all_data_tr['Loan_ID'] = all_data_tr['Loan_ID'].map(lambda x: x[4:]).astype('int')

In [645]:
def family_risk(col):
  
  index = 1

  #Разведённые женщины c детьми и просто неженетые люди живущие в городах / поселках
  if col['Married'] == 'No':
    index *= 2/3
    if (col['Dependents'] != '0') & (col['Gender'] == 'Female'):
      index *= 1/2
      if col['Dependents'] == '2':
        index *= 1.25
      return index

    if (col['Property_Area'] == 'Urban'):
      index *= 0.75

  #Проверка на разведенных
  if (col['Married']=='No') & (col['Dependents']!='0'):
    index *= 0.75
    
  #Почему-то для 2, как правило, больше вероятность отдать
  if col['Dependents'] == '2':
    index *= 1.25


  return index

fr = [family_risk(all_data_tr.iloc[i]) for i in range(len(all_data_tr))]
all_data_tr['Family_Risk'] = fr

In [646]:
all_data_tr['IsDivorced'] = all_data_tr['ApplicantIncome'].map(lambda x: 0)
all_data_tr['IsDivorced'][(all_data_tr['Married']=='No') & (all_data_tr['Dependents']!='0')] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [647]:
family_medium_income = (all_data_tr['ApplicantIncome']+all_data_tr['CoapplicantIncome']) / (1 + all_data_tr['Married'].map({'Yes': 1, 'No': 0})+
              all_data_tr['Dependents'].map({'0': 0, '1': 1, '2': 2, '3+': 3.5}))

all_data_tr['Family_Medium_Income'] = family_medium_income

values = [0, 1700, 2700, 4200, 36000]
labels = ['Low', 'Medium', 'High', 'Very High']

result = pd.cut(all_data_tr['Family_Medium_Income'], bins=values, labels=labels)

all_data_tr['Family_Medium_Income_Flag'] = result

In [648]:
def work_risk(col):
  
  index = 1

  if col['Education'] != 'Graduate':
    index *= 3/4
    if col['Self_Employed'] is 'Yes':
      index *= 2/3

  if col['Education'] == 'Graduate':
    if col['Self_Employed'] is 'Yes':
      index *= 2/3 
  
  # #Для Low - хуже вероятность
  # if col['Family_Medium_Income_Flag'] == 'Low':
  #   index *= 2/3

  #Для Medium - на удивление меньше; для остальных категорий нет разницы
  if col['Family_Medium_Income_Flag'] == 'Medium':
    index *= 1.5

  return index

wr = [work_risk(all_data_tr.iloc[i]) for i in range(len(all_data_tr))]
all_data_tr['Work_Risk'] = wr

In [649]:
all_data_tr['Family_Work_Mean_Risk'] = all_data_tr['Family_Risk'] * 1/2 + all_data_tr['Work_Risk'] * 1/2

In [650]:
def risk(col):
  
  index = col['Family_Work_Mean_Risk']

  if col['Group_Loan_Amount_Term'] == '40 years':
    index *= 0.75

  if col['LoanAmount_Outliers'] == 'Outlier':
    index *= 0.75

  if col['Property_Area'] == 'Rural':
    index *= 2/3
  
  if col['Property_Area'] == 'Semiurban':
    index *= 1.25


  # if col['Credit_History'] == 0:
  #   index *= 0.5

  return index

risk = [risk(all_data_tr.iloc[i]) for i in range(len(all_data_tr))]
all_data_tr['Risk'] = risk

**Теперь быстро за ван-хот энкодим категориональные фичи**

In [651]:
#Для catboost не надо
cat_columns = all_data_tr.select_dtypes(include=['object', 'category']).columns

cat_dummies = pd.DataFrame()
for col in cat_columns:
  cat_dummies = pd.concat([cat_dummies, pd.get_dummies(data=all_data_tr[col], drop_first=True, prefix=col)], axis=1)

**Заскалим все числовые фичи**

In [652]:
num_columns = all_data_tr.select_dtypes(include=np.number).columns

num_final_sc = pd.DataFrame(data=sc.fit_transform(all_data_tr[num_columns]), columns=num_columns)

**Соеденяем дамнисы и числовые и получаем финальный датасет:**

In [653]:
final_data = pd.concat([num_final_sc, cat_dummies], axis=1)
# final_data = pd.concat([num_final_sc, all_data_tr[cat_columns]], axis=1)

In [654]:
# final_data.drop(['Family_Medium_Income'], axis=1, inplace=True)
# final_data.drop(['Group_Loan_Amount_Term_350.0'], axis=1, inplace=True)

**Хит мап**

In [655]:
# final_data_to_train_test = pd.concat([final_data.iloc[:idx, :], target.map({'Y': 0, 'N': 1})], axis=1)

# plt.figure(figsize=(20, 20))
# sns.heatmap(final_data_to_train_test.corr(), annot=True)

**PCA?**

In [656]:
# from sklearn.decomposition import PCA

# pca = PCA()
# pca.fit(final_data)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [657]:
# pca = PCA(n_components=8)
# final_data_pca_tr = pd.DataFrame(data=pca.fit_transform(final_data))

PCA вообще не сработал, даже на казалось бы 20 компонентах

**Most important features by heatmap** 

In [658]:
corr = final_data_to_train_test.corr()['Loan_Status']
feats = corr[np.abs(corr) >= 0.09].index[:-1]

final_data_important = final_data[feats]
len(feats)

8

In [659]:
list(feats)

['Credit_History',
 'Family_Risk',
 'Work_Risk',
 'Family_Work_Mean_Risk',
 'Risk',
 'Married_Yes',
 'Property_Area_Semiurban',
 'Group_Loan_Amount_Term_40 years']

In [660]:
final_data_important = pd.DataFrame(data=final_data[feats], columns=feats)

for col in ['Credit_History','Married_Yes',
            'Property_Area_Semiurban', 'Group_Loan_Amount_Term_40 years']:

  final_data_important[col] = final_data_important[col].astype('int64')

cat_columns_important = final_data_important.select_dtypes(include='int64').columns

Вот это сработало

**Формируем тренировочный / тестовый сет**

In [661]:
# X_train, X_test, y_train, y_test = train_test_split(
#         all_data_tr.iloc[:idx, :], target.map({'Y': 0, 'N': 1}), 
#         stratify=target.map({'Y': 0, 'N': 1}), test_size=0.2, 
#         random_state=RS)

In [662]:
# from lightgbm import LGBMClassifier
# # from sklearn.ensemble import RandomForestClassifier

# clfs = []
# rs = [RS, 34, 43, 69, 101, 123, 144]


# for random_state in rs:

#   X_train, X_test, y_train, y_test = train_test_split(
#         final_data_important.iloc[:idx, :], target.map({'Y': 0, 'N': 1}), 
#         stratify=target.map({'Y': 0, 'N': 1}), test_size=0.2, 
#         random_state=random_state)

#   # clf = LGBMClassifier(n_estimators=150, 
#   #                         max_depth = 10,
#   #                         max_features=7, 
#   #                         random_state=rs)

#   clf = LGBMClassifier(eval_metric='F1',
#                        random_state=random_state)
  
#   clf.fit(X_train, y_train, eval_set=(X_test, y_test),
#                 verbose = 500)
  
#   clfs.append(clf)

#   # clf.fit(final_data_important.iloc[:idx, :], target.map({'Y': 0, 'N': 1}))

#   # clfs.append(clf)

#   # clf.fit(X_train, y_train, eval_set=(X_test, y_test),
#   #               verbose = 500)

# # # lgbm.fit(X_train, y_train)


# # print(lgbm.score(X_test, y_test))
# # print(lgbm.score(X_train, y_train))

# # # print(lgbm.oob_score)

In [663]:
# lst = []
# for clf in clfs:
#   # clf.fit(final_data_important.iloc[:idx, :], target.map({'Y': 0, 'N': 1}))
#   lst.append(clf.predict(final_data_important.iloc[idx:, :]))

# pd.DataFrame(data=lst)

In [664]:
# !pip install catboost

In [665]:
from sklearn.model_selection import KFold # k-фолдная валидация
from catboost import CatBoostClassifier

In [666]:
# clf.predict_proba(all_data_tr.iloc[idx:, :])[:,1]

In [667]:
n_splits = 3


clfs = []
X = final_data_important.iloc[:idx, :]
y = target.map({'Y': 0, 'N': 1})

# параметры валидации, обучение будет идти на 6 X n_splits фолдах

kFold_random_state = [RS , 666, 228, 777, 2021, 75]
N = len(kFold_random_state)*n_splits

for ind_k, random_state in enumerate(kFold_random_state):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_index, test_index in kf.split(X):

        # print(train_index, test_index)

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = CatBoostClassifier(iterations = 10000,
                              loss_function = 'Logloss', eval_metric = 'AUC', #'AUC:hints=skip_train~false',
                              cat_features = list(cat_columns_important), random_seed=random_state, 
                              task_type='CPU', auto_class_weights = 'SqrtBalanced',
                              early_stopping_rounds=250 + ind_k*10 )
        # clfs.append(clf)

        # print(X_train)

        clf.fit(X_train, y_train, eval_set=(X_test, y_test),
                verbose = 500, use_best_model = True, plot = False)
        
        clfs.append(clf)
        
assert len(clfs) == N

# массив для записи финального результата
y_pred = np.zeros((len(all_data_tr.iloc[idx:, :]),))
scores = []
for clf in clfs:

    y_pred += clf.predict_proba(final_data_important.iloc[idx:, :])[:,1]
    scores.append(clf.best_score_['validation']['AUC'])
y_pred /= N
print('mean F1', np.mean(scores))

Learning rate set to 0.009333
0:	test: 0.7363200	best: 0.7363200 (0)	total: 701us	remaining: 7.01s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.7567037279
bestIteration = 15

Shrink model to first 16 iterations.
Learning rate set to 0.009333
0:	test: 0.7390295	best: 0.7390295 (0)	total: 669us	remaining: 6.69s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.805374971
bestIteration = 7

Shrink model to first 8 iterations.
Learning rate set to 0.009338
0:	test: 0.7671860	best: 0.7671860 (0)	total: 583us	remaining: 5.83s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.7866325308
bestIteration = 3

Shrink model to first 4 iterations.
Learning rate set to 0.009333
0:	test: 0.7778592	best: 0.7778592 (0)	total: 627us	remaining: 6.28s
Stopped by overfitting detector  (260 iterations wait)

bestTest = 0.7957365215
bestIteration = 3

Shrink model to first 4 iterations.
Learning rate set to 0.009333
0:	test: 0.7312351	best: 0.731235

In [670]:
len(y_pred[y_pred > 0.47])

120

In [671]:
catboost_params = { 'iterations' : 5000, 'depth' : 8,
                   'learning_rate' : 0.13851548141121092/8,
                   'colsample_bylevel' : 0.42513600460183565/8,
                   'bagging_temperature' : 3.7987780158429185/8,
                   'early_stopping_rounds': 500/2, 'random_seed' : 42,
                   'loss_function' : 'Logloss','eval_metric' : 'Accuracy',
                   'boosting_type' : 'Plain','bootstrap_type' : 'MVS',
                   'task_type' : 'CPU', 'cat_features' : list(cat_columns_important)}

In [672]:
n_splits = 5

X = final_data_important.iloc[:idx, :]
y = target.map({'Y': 0, 'N': 1})

optuna_clfs = []
kFold_random_state = [101]#, 42, 666, 228]#, 777, 2021, 75]
N = len(kFold_random_state)*n_splits

for ind_k, random_state in enumerate(kFold_random_state):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf = CatBoostClassifier(**catboost_params)
        optuna_clfs.append(clf)

        clf.fit(X_train, y_train, eval_set=(X_test, y_test),
                verbose=500, use_best_model = True, plot=False)


# массив для записи результата по нескольким фолдам
y_pred_optuna = np.zeros((len(final_data_important.iloc[idx:, :]),))
scores = []
for clf in optuna_clfs:
    y_pred_optuna += clf.predict_proba(final_data_important.iloc[idx:, :])[:,1]
    scores.append(clf.best_score_['validation']['Accuracy'])
y_pred_optuna /= N
print('mean AUC', np.mean(scores))

0:	learn: 0.7006110	test: 0.6341463	best: 0.6341463 (0)	total: 434us	remaining: 2.17s
500:	learn: 0.8248473	test: 0.8048780	best: 0.8048780 (458)	total: 125ms	remaining: 1.12s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.8048780488
bestIteration = 458

Shrink model to first 459 iterations.
0:	learn: 0.6945010	test: 0.6585366	best: 0.6585366 (0)	total: 463us	remaining: 2.32s
500:	learn: 0.8289206	test: 0.7886179	best: 0.7886179 (254)	total: 136ms	remaining: 1.22s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.7886178862
bestIteration = 254

Shrink model to first 255 iterations.
0:	learn: 0.6782077	test: 0.6991870	best: 0.6991870 (0)	total: 400us	remaining: 2s
500:	learn: 0.8187373	test: 0.8455285	best: 0.8455285 (265)	total: 121ms	remaining: 1.09s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.8455284553
bestIteration = 265

Shrink model to first 266 iterations.
0:	learn: 0.6863544	test: 0.6910569	best: 0.6910569 (0)	to

In [690]:
t = np.array(y_pred_optuna * 0.85 + y_pred * 0.15)
t

array([0.3310765 , 0.3310765 , 0.3177824 , 0.3177824 , 0.38568012,
       0.34711234, 0.3646594 , 0.60924845, 0.32894979, 0.3269675 ,
       0.38568012, 0.2881172 , 0.37938492, 0.53314662, 0.32685382,
       0.37938492, 0.29018421, 0.2881172 , 0.33737305, 0.32685382,
       0.34711234, 0.40688301, 0.367757  , 0.38568012, 0.33737305,
       0.62176477, 0.3177824 , 0.33737305, 0.27265586, 0.367757  ,
       0.32202146, 0.29018421, 0.32685382, 0.3310765 , 0.29348176,
       0.57605997, 0.3177824 , 0.29348176, 0.367757  , 0.367757  ,
       0.3310765 , 0.367757  , 0.28675771, 0.32685382, 0.367757  ,
       0.3310765 , 0.34711234, 0.3269675 , 0.367757  , 0.30778165,
       0.3310765 , 0.30778165, 0.27265586, 0.3177824 , 0.32503374,
       0.6234382 , 0.37786108, 0.27061985, 0.53314662, 0.2881172 ,
       0.28675771, 0.3310765 , 0.34197382, 0.62176477, 0.34711234,
       0.2881172 , 0.53360274, 0.5127836 , 0.2881172 , 0.60509797,
       0.32685382, 0.32685382, 0.29348176, 0.34467729, 0.30778

In [ ]:
y_pred

In [349]:
len(y_pred[y_pred >=  0.5])

59

In [236]:
# print(clfs[0].score(X_train, y_train))
# print(clfs[1].score(X_train, y_train))
# print(clfs[2].score(X_train, y_train))


# print(clfs[0].score(X_test, y_test))
# print(clfs[1].score(X_test, y_test))
# print(clfs[2].score(X_test, y_test))

In [237]:
# print(classification_report(y_train, clfs[0].predict(X_train)))
# print(confusion_matrix(y_train, clfs[0].predict(X_train)))
# # print(clfs[1].score(X_train, y_train))
# # print(clfs[2].score(X_train, y_train))


# print(classification_report(y_test, clfs[0].predict(X_test)))
# print(confusion_matrix(y_test, clfs[0].predict(X_test)))
# # print(clfs[1].score(X_test, y_test))
# # print(clfs[2].score(X_test, y_test))

In [291]:
# lst = []
# for clf in clfs:
#   lst.append(clf.predict(all_data_tr.iloc[idx:, :]))

# df = pd.DataFrame(data=lst).T

# # idx0, idx1, idx2, idx3 = 0, 0, 0, 0 

# lst0 = [1 for i in range(367) if sum(df.iloc[i].values) == 0]
# lst1 = [1 for i in range(367) if sum(df.iloc[i].values) == 1]
# lst2 = [1 for i in range(367) if sum(df.iloc[i].values) == 2]
# lst3 = [1 for i in range(367) if sum(df.iloc[i].values) == 3]
# # lst4 = [1 for i in range(367) if sum(df.iloc[i].values) == 4]
# # lst5 = [1 for i in range(367) if sum(df.iloc[i].values) == 5]

# print(len(lst0))
# print(len(lst1))
# print(len(lst2))
# print(len(lst3))
# # print(len(lst4))
# # print(len(lst5))
# # lst2 = []
# # lst3 = []

In [269]:
# import scipy 

# ct_predict = np.array(scipy.stats.mode(lst)[0][0])

In [292]:
# print(confusion_matrix(y_test, lgbm.predict(X_test)))
# print(classification_report(y_test, lgbm.predict(X_test)))

In [293]:
# print(confusion_matrix(y_train, lgbm.predict(X_train)))
# print(classification_report(y_train, lgbm.predict(X_train)))

In [ ]:
# plt.figure(figsize=(70, 10))

# sns.barplot(final_data.columns, lgbm.feature_importances_)

Лог регрессия предсказывает настоящие нули с вероятностью 0.5...
Xa-xa-xa!

In [ ]:
# lr_predict = lr.predict(final_data_pca_tr.iloc[idx:, :])

# lr_predict = np.where(lr_predict == 0, 'Y', lr_predict)
# lr_predict = np.where(lr_predict == 1, 'N', lr_predict)

In [ ]:
# lgbm_predict = lgbm.predict(final_data_important.iloc[idx:, :])

# lgbm_predict = np.where(lgbm_predict == 0, 'Y', lgbm_predict)
# lgbm_predict = np.where(lgbm_predict == '1', 'N', lgbm_predict)

In [691]:
t = np.where(t < 0.43, 0, t)
t = np.where(t >= 0.43, 1, t)

# t.astype('object')

t = np.where(t == 0, 'Y', t)
t = np.where(t == '1.0', 'N', t)

In [692]:
t

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [695]:
sample_submission['Loan_Status'] = t

In [696]:
sample_submission.to_csv('sub2_26.csv', index=False)

In [ ]:
# train_tr.to_csv('Train_tr.csv', index=False)
# test_tr.to_csv('Test_tr.csv', index=False)

# target.to_csv('Target.csv', index=False)